In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split

In [10]:
Data = pd.read_csv('fulldata.csv')

In [11]:
selected_features = ['diag', 'pca1', 'pca2', 'pca3']
X = Data[selected_features]
target = ['energy']
y = Data[target]

In [12]:
from sklearn.preprocessing import MaxAbsScaler

max_abs_scaler = MaxAbsScaler()
X_scale = max_abs_scaler.fit_transform(X)
X_scale

array([[ 0.378     ,  0.84518036, -0.53982263,  0.06834789],
       [ 0.005     ,  0.46454217,  0.64732854, -0.54425677],
       [ 0.005     ,  0.26547058,  0.54895332,  0.56411071],
       ...,
       [ 0.005     ,  0.02825252, -0.00129901, -0.03025052],
       [ 0.005     ,  0.0241501 ,  0.00616268, -0.01113919],
       [ 0.005     ,  0.02129546, -0.00274642, -0.01495718]])

In [13]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X_scale, y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.33, random_state=42)

In [14]:
from scipy.stats import randint, uniform

def create_model(units1=48, units2=48, units3=32, units4=32, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(Dense(units=units1, input_dim=4, activation=activation))
    model.add(Dense(units=units2, activation=activation))
    model.add(Dense(units=units3, activation=activation))
    model.add(Dense(units=units4, activation=activation))
    model.add(Flatten())
    model.add(Dense(units=1))

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

    
model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=32, verbose=0)
param_distributions = {
    'units1': randint(32, 64, 128),
    'units2': randint(32, 64, 128),
    'units3': randint(16, 32, 64),
    'units4': randint(16, 32, 64),
    'activation': ['tanh', 'relu', 'swish'],
    'optimizer': ['adam']
}

C:\Users\gts\AppData\Local\Temp\ipykernel_20756\3403441553.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=32, verbose=0)


In [15]:
random_search = RandomizedSearchCV(
    estimator=model, param_distributions=param_distributions,
    n_iter=10, cv=3, n_jobs=-1, verbose=2)

In [16]:
random_search_result = random_search.fit(X_train, Y_train, validation_data=(X_val, Y_val))


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))
means = random_search_result.cv_results_['mean_test_score']
stds = random_search_result.cv_results_['std_test_score']
params = random_search_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
best_model = random_search_result.best_estimator_.model
test_loss, test_mae = best_model.evaluate(X_test, Y_test)
print(f"Test loss: {test_loss:.3f}, Test MAE: {test_mae:.3f}")